In [233]:
#Irish M50 Highway Dataset  
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
from pyspark.sql.functions import col,udf,lit
from pyspark.sql.types import StringType
df = spark.read.csv('/home/bhavyamilanshukla/per-vehicle-records-2020-01-31.csv',
inferSchema = True, header = True)
count = df.count()
m50=[15012,15011,15010,1507,1506,1505,1504,1509,1503,1508,1502,1501,1500,1012]

In [234]:
# Question 1
df.groupBy("classname").count().withColumn("Percentage",col("count")/count*100).show()

+---------+-------+--------------------+
|classname|  count|          Percentage|
+---------+-------+--------------------+
|      CAR|3804948|   80.25858594040196|
|  HGV_ART| 208477|  4.3974501678070705|
|      BUS|  32575|  0.6871114761643508|
|  HGV_RIG| 129477|  2.7310861887745705|
|     null|    347|0.007319345578788325|
|  CARAVAN|  20344|  0.4291203644232556|
|      LGV| 530714|  11.194464465420943|
|    MBIKE|  13979| 0.29486205142905475|
+---------+-------+--------------------+



In [278]:
# Question 2
x = df.filter(col("cosit").isin(m50))
x.groupBy("hour").count().sort("count",ascending=False).show()


+----+-----+
|hour|count|
+----+-----+
|   7|67199|
|  14|66812|
|   8|65832|
|  16|65370|
|  17|59487|
|  12|56581|
|   9|55841|
|  10|53002|
|  13|51568|
|  18|51192|
|  11|47798|
|   6|42262|
|  15|40698|
|  19|39545|
|  20|30234|
|  21|21773|
|  22|16428|
|   5|12235|
|  23|12063|
|   0| 5811|
+----+-----+
only showing top 20 rows



In [275]:
# Question 3 # Morning
df.filter((col("cosit").isin(m50))&(df["hour"] >= 6)&(df["hour"] <= 10)).groupBy("hour").count().withColumn("Time Of Day",lit("Morning")).show()

+----+-----+-----------+
|hour|count|Time Of Day|
+----+-----+-----------+
|   6|42262|    Morning|
|   9|55841|    Morning|
|   8|65832|    Morning|
|   7|67199|    Morning|
|  10|53002|    Morning|
+----+-----+-----------+



In [237]:
# Question 3 # Evening
df.filter((col("cosit").isin(m50))&(df["hour"] >= 16)&(df["hour"] <= 20)).groupBy("hour").count().withColumn("Time Of Day",lit("Evening")).show()

+----+-----+-----------+
|hour|count|Time Of Day|
+----+-----+-----------+
|  16|65370|    Evening|
|  20|30234|    Evening|
|  19|39545|    Evening|
|  17|59487|    Evening|
|  18|51192|    Evening|
+----+-----+-----------+



In [239]:
# Question 4 
x=df.filter(col("cosit").isin(m50)).groupBy('cosit').avg('speed')
names={15012:'j16-j17',15011:'j15-j16',15010:'j14-j15',1507:'j13-j14',1506:'j12-j13',1505:'j11-j12',1504:'j10-j11',1509:'j9-j10',1503:'j7-j9',1508:'j6-j7',1502:'j5-j6',1501:'j4-j5',1500:'j3-j4',1012:'j2-j3'}
map_value=udf(lambda x:names[x], StringType())
x.withColumn('junction',map_value("cosit")).show()

+-----+-----------------+--------+
|cosit|       avg(speed)|junction|
+-----+-----------------+--------+
| 1507|99.00845921450151| j13-j14|
| 1500| 79.3267028425403|   j3-j4|
| 1506|90.89604001118734| j12-j13|
| 1505|85.31515519064502| j11-j12|
| 1504|84.48510440700593| j10-j11|
|15010|97.81252274326192| j14-j15|
|15012|93.78990503959967| j16-j17|
| 1509|78.43267256357605|  j9-j10|
| 1502|87.70600667675384|   j5-j6|
|15011|97.19775581634754| j15-j16|
| 1501|87.25861183179634|   j4-j5|
| 1503| 81.0891341051616|   j7-j9|
| 1508|82.67599094114848|   j6-j7|
| 1012| 78.5138596978229|   j2-j3|
+-----+-----------------+--------+



In [229]:
# Question 5
x=df.filter(col("classname").isin(["HGV_ART","HGV_RIG"])).groupBy('cosit').count().sort("count",ascending=False).limit(10)
names={997:'Test',1508:'Castleknock',1015:'Donabate',200723:'Kilteel',1502:'Finglas',1501:'Ballymun',1503:'Palmerstown',1071:'Kingswood',1070:'Newlands Cross',1073:'VMS Citywest'}
map_value=udf(lambda x:names[x], StringType())
x.withColumn('location',map_value("cosit")).show()

+------+-----+--------------+
| cosit|count|      location|
+------+-----+--------------+
|   997|11986|          Test|
|  1508| 9630|   Castleknock|
|  1015| 9462|      Donabate|
|200723| 8977|       Kilteel|
|  1502| 7922|       Finglas|
|  1501| 7145|      Ballymun|
|  1503| 6938|   Palmerstown|
|  1071| 6259|     Kingswood|
|  1070| 5850|Newlands Cross|
|  1073| 5846|  VMS Citywest|
+------+-----+--------------+

